In [14]:
import datetime
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import time
import lightgbm
from itertools import product
from sklearn.preprocessing import LabelEncoder
%matplotlib inline

In [15]:
items = pd.read_csv('../sale-prediction/items.csv')
shops = pd.read_csv('../sale-prediction/shops.csv')
cats = pd.read_csv('../sale-prediction/item_categories.csv')
sales = pd.read_csv('../sale-prediction/sales_train.csv')
test = pd.read_csv('../sale-prediction/test.csv').set_index('ID')

In [16]:
# Якутск Орджоникидзе, 56
train.loc[train.shop_id == 0, 'shop_id'] = 57
test.loc[test.shop_id == 0, 'shop_id'] = 57
# Якутск ТЦ "Центральный"
train.loc[train.shop_id == 1, 'shop_id'] = 58
test.loc[test.shop_id == 1, 'shop_id'] = 58
# Жуковский ул. Чкалова 39м²
train.loc[train.shop_id == 10, 'shop_id'] = 11
test.loc[test.shop_id == 10, 'shop_id'] = 11
#かぶっている店の名前を変更。ほかにもあって気がするが。
shops.loc[shops.shop_name == 'Сергиев Посад ТЦ "7Я"', 'shop_name'] = 'СергиевПосад ТЦ "7Я"'
shops['city'] = shops['shop_name'].str.split(' ').map(lambda x: x[0])
shops.loc[shops.city == '!Якутск', 'city'] = 'Якутск'
shops['city_code'] = LabelEncoder().fit_transform(shops['city'])
shops = shops[['shop_id','city_code']] #この二つだけにしちゃうんだ
cats['split'] = cats['item_category_name'].str.split('-')
cats['type'] = cats['split'].map(lambda x: x[0].strip())
cats['type_code'] = LabelEncoder().fit_transform(cats['type'])
cats['subtype'] = cats['split'].map(lambda x: x[1].strip() if len(x) > 1 else x[0].strip())
cats['subtype_code'] = LabelEncoder().fit_transform(cats['subtype'])
cats = cats[['item_category_id', 'type_code', 'subtype_code']]
items.drop(['item_name'], axis=1, inplace=True)

In [25]:
train = sales.join(items, on='item_id', rsuffix='_')
train = train.join(shops, on='shop_id', rsuffix='_')
train = train.join(cats, on='item_category_id', rsuffix='_')
train = train.drop(['item_id_', 'shop_id_', 'item_category_id_'], axis=1)
train

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_category_id,city_code,type_code,subtype_code
0,02.01.2013,0,59,22154,999.00,1.0,37,30,11,1
1,03.01.2013,0,25,2552,899.00,1.0,58,13,13,27
2,05.01.2013,0,25,2552,899.00,-1.0,58,13,13,27
3,06.01.2013,0,25,2554,1709.05,1.0,58,13,13,27
4,15.01.2013,0,25,2555,1099.00,1.0,56,13,13,3
...,...,...,...,...,...,...,...,...,...,...
2935844,10.10.2015,33,25,7409,299.00,1.0,55,13,13,2
2935845,09.10.2015,33,25,7460,299.00,1.0,55,13,13,2
2935846,14.10.2015,33,25,7459,349.00,1.0,55,13,13,2
2935847,22.10.2015,33,25,7440,299.00,1.0,57,13,13,8


In [26]:
train = train.query('item_price > 0')

In [27]:
train.columns

Index(['date', 'date_block_num', 'shop_id', 'item_id', 'item_price',
       'item_cnt_day', 'item_category_id', 'city_code', 'type_code',
       'subtype_code'],
      dtype='object')

In [38]:
train_monthly = train.groupby(['date_block_num', 'shop_id', 'item_id']).agg({'item_cnt_day': ['sum']})
train_monthly.columns = ['item_cnt_month']
train_monthly.reset_index(inplace=True)
train_monthly = train_monthly.join(items, on='item_id', rsuffix='_').join(shops, on='shop_id', rsuffix='_').join(cats, on='item_category_id', rsuffix='_')

## matrix再構成

In [49]:
shop_ids = train_monthly['shop_id'].unique()
item_ids = train_monthly['item_id'].unique()
empty_df = []
for i in range(34):
    for shop in shop_ids:
        for item in item_ids:
            empty_df.append([i, shop, item])
    
empty_df = pd.DataFrame(empty_df, columns=['date_block_num','shop_id','item_id'])

In [50]:
train_monthly = pd.merge(empty_df, train_monthly, on=['date_block_num','shop_id','item_id'], how='left')
train_monthly.fillna(0, inplace=True)
train_monthly

,date_block_num,shop_id,item_id,item_cnt_month,item_id_,item_category_id,shop_id_,city_code,item_category_id_,type_code,subtype_code
0,0,59,22154,1.0,22154.0,37.0,59.0,30.0,37.0,11.0,1.0
1,0,59,2552,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,59,2554,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,59,2555,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,59,2564,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
44486275,33,36,8428,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
44486276,33,36,7903,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
44486277,33,36,7610,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
44486278,33,36,7635,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Test

In [51]:
test['date_block_num'] = 34
test = test.join(items, on='item_id', rsuffix='_').join(shops, on='shop_id', rsuffix='_').join(cats, on='item_category_id', rsuffix='_')

In [52]:
test

,shop_id,item_id,date_block_num,item_id_,item_category_id,shop_id_,city_code,item_category_id_,type_code,subtype_code
ID,,,,,,,,,,
0,5,5037,34,5037,19,5,3,19,5,10
1,5,5320,34,5320,55,5,3,55,13,2
2,5,5233,34,5233,19,5,3,19,5,10
3,5,5232,34,5232,23,5,3,23,5,16
4,5,5268,34,5268,20,5,3,20,5,11
...,...,...,...,...,...,...,...,...,...,...
214195,45,18454,34,18454,55,45,20,55,13,2
214196,45,16188,34,16188,64,45,20,64,14,42
214197,45,15757,34,15757,55,45,20,55,13,2


In [55]:
matrix = pd.concat([train_monthly, test],ignore_index=True, sort=False)
matrix

,date_block_num,shop_id,item_id,item_cnt_month,item_id_,item_category_id,shop_id_,city_code,item_category_id_,type_code,subtype_code
0,0,59,22154,1.0,22154.0,37.0,59.0,30.0,37.0,11.0,1.0
1,0,59,2552,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,59,2554,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,59,2555,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,59,2564,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
44700475,34,45,18454,NaN,18454.0,55.0,45.0,20.0,55.0,13.0,2.0
44700476,34,45,16188,NaN,16188.0,64.0,45.0,20.0,64.0,14.0,42.0
44700477,34,45,15757,NaN,15757.0,55.0,45.0,20.0,55.0,13.0,2.0
44700478,34,45,19648,NaN,19648.0,40.0,45.0,20.0,40.0,11.0,4.0
